In [ ]:
import pandas as pd

#Load the dataset
df = pd.read_csv('auc_results.csv')

sampled_curves = {}
for curve_type in df['curve_type'].unique():
    sampled_curves[curve_type] = df[df['curve_type'] == curve_type]['parameters'].to_list()


In [ ]:
#1. Visualize the zoo of response curves

import numpy as np
import matplotlib.pyplot as plt
from response_curves import generate_response_curve
import ast

# Create time points
x = np.linspace(0, 200, 500)

# Create a figure for plotting
plt.figure(figsize=(12, 16))

# Generate and plot different response curves
curve_types = ['skewed_gaussian', 'biexponential', 'gamma', 'lognormal', 
               'weibull', 'bateman', 'inverse_gaussian']

for i, curve_type in enumerate(curve_types):

    plt.subplot(len(curve_types), 1, i+1)
    plt.title(f"{curve_type.replace('_', ' ').capitalize()}")

    for params in sampled_curves[curve_type]:
        params = ast.literal_eval(params)
        response, start, end = generate_response_curve(x, curve_type=curve_type, params=params)

        # Plot
        plt.plot(x, response)
        plt.grid(True, alpha=0.3)
        ax = plt.gca()
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

plt.xlabel('Time (min)', fontsize=12)
plt.gcf().text(-.01, 0.5, 'Density', va='center', rotation='vertical', fontsize=12)

plt.tight_layout()
plt.savefig('response_curves_zoo.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Define the number of rows for the grid
plt.rcParams.update({'font.size': 14})

num_rows = max(len(range(5, 42, 5)), len(range(4, 11)), len(df['curve_type'].unique()))

# Create a figure and axes for the grid
fig, axes = plt.subplots(num_rows, 3, figsize=(21, num_rows * 4), sharex=True, sharey=True)

# Flatten the axes array for easier indexing
axes = axes.flatten()

num_bins = 50

# Plot participant histograms in the left column
for i, num_participants in enumerate(range(5, 42, 5)):
    sub_df = df[df['participants'] == num_participants]  # Filter the DataFrame for the current number of participants
    
    ax = axes[i * 3]  # Left column
    sub_df['standard_auc_mean'].hist(bins=num_bins, alpha=0.7, label='Standard AUC Mean', color='blue', ax=ax, density=True)
    sub_df['mc_auc_mean'].hist(bins=num_bins, alpha=0.7, label='Monte Carlo AUC Mean', color='orange', ax=ax, density=True)
    
    ax.set_xlabel('AUC Value', fontsize=12)
    ax.set_title(f'Participants = {num_participants}', fontsize=14)
    ax.legend(fontsize=14)
    ax.grid(alpha=0.3)

# Plot datapoint histograms in the middle column
for i, num_datapoints in enumerate(range(4, 11)):
    sub_df = df[df['grid_points_in_graph'] == num_datapoints]  # Filter the DataFrame for the current number of datapoints
    
    ax = axes[i * 3 + 1]  # Middle column
    sub_df['standard_auc_mean'].hist(bins=num_bins, alpha=0.7, label='Standard AUC Mean', color='blue', ax=ax, density=True)
    sub_df['mc_auc_mean'].hist(bins=num_bins, alpha=0.7, label='Monte Carlo AUC Mean', color='orange', ax=ax, density=True)
    
    ax.set_xlabel('AUC Value', fontsize=12)
    ax.set_title(f'Data Points = {num_datapoints}', fontsize=14)
    ax.grid(alpha=0.3)

# Plot curve_type histograms in the right column
for i, curve_type in enumerate(df['curve_type'].unique()):
    sub_df = df[df['curve_type'] == curve_type]  # Filter the DataFrame for the current curve_type
    
    ax = axes[i * 3 + 2]  # Right column
    sub_df['standard_auc_mean'].hist(bins=num_bins, alpha=0.7, label='Standard AUC Mean', color='blue', ax=ax, density=True)
    sub_df['mc_auc_mean'].hist(bins=num_bins, alpha=0.7, label='Monte Carlo AUC Mean', color='orange', ax=ax, density=True)
    
    ax.set_xlabel('AUC Value', fontsize=12)
    ax.set_title(f"{curve_type.replace('_', ' ').capitalize()}",fontsize=14)

    ax.grid(alpha=0.3)


ax = axes[-1]  # Left column  
df['standard_auc_mean'].hist(bins=num_bins, alpha=0.7, label='Standard AUC Mean', color='blue',density=True)
df['mc_auc_mean'].hist(bins=num_bins, alpha=0.7, label='Monte Carlo AUC Mean', color='orange',density=True)
ax.set_xlabel('AUC Value', fontsize=12)
ax.set_title(f'All Curves', fontsize=14)
ax.grid(alpha=0.3)  
   
for i, ax in enumerate(axes):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    if i > 0:
        ax.legend().remove()
    ax.plot([1,1], [0,17], 'k--')

# Remove x-axis labels for all but the bottom row of plots
for i, ax in enumerate(axes[:-3]):  # Exclude the last row of plots
    if i != 19:
        ax.set_xlabel('')

for ax in axes:
    ax.set_xlim(0.7, 1.3)

# Clear the second to last axis (last row, middle column)
axes[-2].clear()
axes[-2].set_visible(False)

plt.gcf().text(-.01, 0.5, 'Density', va='center', rotation='vertical', fontsize=12)

# Adjust layout
plt.tight_layout()
plt.savefig('auc_comparison_results.png', dpi=1200, bbox_inches='tight')

plt.savefig('auc_comparison_results.pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Visualize the results
import matplotlib.pyplot as plt


# Create the bar plot
x_labels = df["grid_points_in_graph"].astype(str) + "_" + df["participants"].astype(str)
x = np.arange(len(x_labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(12, 6))

# Plot the bars
bar1 = ax.bar(x - width/2, df["standard_auc_mean"], width, yerr=df["standard_auc_simple_std"], label='Standard AUC', capsize=5)
bar2 = ax.bar(x + width/2, df["mc_auc_mean"], width, yerr=df["mc_auc_std"], label='Monte Carlo AUC', capsize=5)

# Add labels, title, and legend
ax.set_xlabel('Grid Points and Participants')
ax.set_ylabel('AUC')
ax.set_title('Comparison of Standard AUC and Monte Carlo AUC')
ax.set_xticks(x)
ax.set_xticklabels(x_labels, rotation=45, ha='right')
ax.legend()

start, end = ax.get_xlim()

ax.plot([start, end], [1, 1], 'k--', label='True AUC', color='black')
ax.set_xlim(-0.5, end-1)
# Show the plot
plt.tight_layout()
plt.savefig('auc_comparison.pdf', dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
# Calculate mc_coverage percentage per curve_type
mc_coverage_percentage = df.groupby('curve_type')['mc_coverage'].mean() * 100
standard_coverage_percentage = df.groupby('curve_type')['standard_coverage'].mean() * 100

# Display the result
print("Standard curve_type")
print(df.groupby('curve_type')['standard_coverage'].mean() * 100)
print("\nMonte Carlo curve_type")
print(df.groupby('curve_type')['mc_coverage'].mean() * 100)
